In [1]:
import pandas as pd
import gmaps
import requests 
from config import gkey


In [2]:
#pull in city data from csv file
city_data = pd.read_csv("Weather_data/cities.csv")
city_data.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Puro,PH,2022-07-23 13:21:44,12.4841,123.3827,82.63,77,77,3.69
1,1,Yellowknife,CA,2022-07-23 13:21:44,62.4560,-114.3525,63.12,72,75,13.80
2,2,Broome,US,2022-07-23 13:21:45,42.2506,-75.8330,78.48,74,0,8.05
3,3,Chitral,PK,2022-07-23 13:21:45,35.8462,71.7858,83.01,62,97,7.47
4,4,Vaini,TO,2022-07-23 13:21:45,-21.2000,-175.2000,69.96,83,20,9.22


In [3]:
city_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     590 non-null    int64  
 1   City        590 non-null    object 
 2   Country     586 non-null    object 
 3   Date        590 non-null    object 
 4   Lat         590 non-null    float64
 5   Lng         590 non-null    float64
 6   Max Temp    590 non-null    float64
 7   Humidity    590 non-null    int64  
 8   Cloudiness  590 non-null    int64  
 9   Wind Speed  590 non-null    float64
dtypes: float64(4), int64(3), object(3)
memory usage: 46.2+ KB


In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=gkey)

In [11]:
#Assign the array of latitude and longitude pairs
locations = city_data[["Lat", "Lng"]]
max_temps=city_data["Max Temp"]
# Get the maximum temperature.
max_temp = city_data["Max Temp"]

#Assign a figure variable
fig=gmaps.figure()
#Create heatmap variable
heat_layer=gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],dissipating=False, max_intensity=300, point_radius=4)
#Add the heatmap layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of percent humidity
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Heatmap of percent cloudiness 
locations = city_data[["Lat", "Lng"]]
clouds = city_data["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Heatmap of wind speed 
locations = city_data[["Lat", "Lng"]]
wind = city_data["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [25]:
preferred_cities = city_data.loc[(city_data["Max Temp"] <= max_temp) & (city_data["Max Temp"]>= min_temp)]
preferred_cities.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Puro,PH,2022-07-23 13:21:44,12.4841,123.3827,82.63,77,77,3.69
3,3,Chitral,PK,2022-07-23 13:21:45,35.8462,71.7858,83.01,62,97,7.47
22,22,Bethel,US,2022-07-23 13:20:15,41.3712,-73.4140,84.42,79,0,5.75
25,25,Hithadhoo,MV,2022-07-23 13:21:51,-0.6000,73.0833,81.39,77,99,8.75
35,35,Volodarskiy,RU,2022-07-23 13:21:54,46.4013,48.5409,86.99,25,99,14.94


In [26]:
preferred_cities.count()

City_ID       135
City          135
Country       135
Date          135
Lat           135
Lng           135
Max Temp      135
Humidity      135
Cloudiness    135
Wind Speed    135
dtype: int64

In [27]:
hotel_df= preferred_cities[["City","Country","Max Temp","Lat","Lng"]].copy()
hotel_df["Hotel Name"]=''
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Puro,PH,82.63,12.4841,123.3827,
3,Chitral,PK,83.01,35.8462,71.7858,
22,Bethel,US,84.42,41.3712,-73.4140,
25,Hithadhoo,MV,81.39,-0.6000,73.0833,
35,Volodarskiy,RU,86.99,46.4013,48.5409,


In [31]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

    

In [32]:
#iterate through the dataframe
for index,row in hotel_df.iterrows():
    #get the latitude and longitude
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels=requests.get(base_url,params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"]=hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found....skipping.")

Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.


In [40]:
hotel_df.head(50)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Puro,PH,82.63,12.4841,123.3827,
3,Chitral,PK,83.01,35.8462,71.7858,PTDC Motel Chitral
22,Bethel,US,84.42,41.3712,-73.4140,Courtyard by Marriott Danbury
25,Hithadhoo,MV,81.39,-0.6000,73.0833,Scoop Guest House
35,Volodarskiy,RU,86.99,46.4013,48.5409,Gostinyy Dvorik
37,Yuli,NG,80.51,9.6989,10.2744,
39,Cayenne,GF,87.10,4.9333,-52.3333,Hôtel Le Dronmi
45,Albany,US,80.42,42.6001,-73.9662,
52,Touros,BR,81.79,-5.1989,-35.4608,INN NEW HORIZON
57,Kralendijk,BQ,85.98,12.1500,-68.2667,Divi Flamingo Beach Resort & Casino


In [44]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))